# Polynomial Linear Regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Exploratory Data Analysis and Visualization

In [ ]:
tv = np.array([181,9,58,120,9,200,66,215,24,98,204,195,68,281,69,147,218,237,13,228,62,263,143,240,249])
radio = np.array([11,49,33,20,2,3,6,24,35,8,33,48,37,40,21,24,28,5,16,17,13,4,29,17,27])
newspaper = np.array([58,75,24,12,1,21,24,4,66,7,46,53,114,56,18,19,53,24,50,26,18,20,13,23,23])
sales = np.array([13,7,12,13,5,11,9,17,9,10,19,22,13,24,11,15,18,13,6,16,10,12,15,16,19])

df = pd.DataFrame({'tv': tv, 'radio': radio, 'newspaper': newspaper, 'sales': sales})
df.head()

## Data Preprocessing

In [ ]:
X = df.drop('sales', axis=1) # keep only the features
y = df['sales']

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
polynomial_converter = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
# Converter "fits" to data, in this case, reads in every X column
# Then it "transforms" and ouputs the new polynomial data
poly_features = polynomial_converter.fit_transform(X)

In [ ]:
poly_features.shape

In [ ]:
X.shape

In [ ]:
X.iloc[0]

In [ ]:
# the first 3 items are our original items from X
# the 4th item is square of feature1 (181 * 181)
# the 5th item feature1 * feature2 (181 * 11)
# the 6th item feature1 * feature3 (181 * 58)
# the 7th item is square of feature2 (11 * 11)
# the 8th item feature2 * feature3 (11 * 58)
# the 9th item is square of feature3 (58 * 58)
poly_features[0]

Now after on base of our 3 initial features we have 9 features and we will use this new polynomial regression set

## Train and Test Splits

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# we are now predicting y (sales) based on our new dataset with 9 features
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=99)

## Creating the model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression(fit_intercept=True)

In [ ]:
model.fit(X_train, y_train)

## Predictions on the test data

We want to fairly evaluate our model, so we get performance metrics on the test set (data the model has never seen before).

In [ ]:
y_pred = model.predict(X_test)
y_pred

## Model Performance

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)

In [ ]:
print(MAE)
print(MSE)
print(RMSE)
print(df['sales'].mean())

### Comparison with Simple Linear Regression

Results on the Test Set (Note: Use the same Random Split to fairly compare)

- Simple Linear Regression:
  - MAE: 2.0326719009660215
  - RMSE: 2.391906607310042

- Polynomial 2-degree:
  - MAE: 0.9084497157236153
  - RMSE: 1.1193077657280737
  
We can see that the polynomial regression is performing better.

## Overfitting and Underfitting

### Overfitting

- model will perform very well on training data, but will have poor performance on new unseen data

### Underfitting

- model does not capture the underlying trend of the data and does not fit the data well enough
- can lead to poor performance in both training and testing data sets

## Choosing a Model

### Adjusting Parameters

Are we satisfied with this performance? Perhaps a higher order would improve performance even more! But how high is too high? It is now up to us to possibly go back and adjust our model and parameters, let's explore higher order Polynomials in a loop and plot out their error. This will nicely lead us into a discussion on Overfitting.

Let's use a for loop to do the following:

1. Create different order polynomial X data
2. Split that polynomial data for train/test
3. Fit on the training data
4. Report back the metrics on *both* the train and test results
5. Plot these results and explore overfitting

In [ ]:
# TRAINING ERROR PER DEGREE
train_rmse_errors = []
# TEST ERROR PER DEGREE
test_rmse_errors = []

for d in range(1, 10):
    # CREATE POLY DATA SET FOR DEGREE "d"
    polynomial_converter = PolynomialFeatures(degree=d, include_bias=False)
    poly_features = polynomial_converter.fit_transform(X)
    
    # SPLIT THIS NEW POLY DATA SET
    X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=99)
    
    # TRAIN ON THIS NEW POLY SET
    model = LinearRegression(fit_intercept=True)
    model.fit(X_train, y_train)
    
    # PREDICT ON BOTH TRAIN AND TEST
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    # Calculate Errors
    
    # Errors on Train Set
    train_RMSE = np.sqrt(mean_squared_error(y_train, train_pred))
    
    # Errors on Test Set
    test_RMSE = np.sqrt(mean_squared_error(y_test, test_pred))

    # Append errors to lists for plotting later
    
   
    train_rmse_errors.append(train_RMSE)
    test_rmse_errors.append(test_RMSE)

In [ ]:
train_rmse_errors

In [ ]:
test_rmse_errors

In [ ]:
plt.plot(range(1, 10), train_rmse_errors, label='TRAIN RMSE')
plt.plot(range(1, 10), test_rmse_errors, label='TEST RMSE')
plt.xlabel("Polynomial Complexity")
plt.ylabel("RMSE")
plt.legend()

Looking at the plot below it looks like the best `degree` value for us is 2, after that things are getting really far.

In [ ]:
# same as above, but in more focused on the closer part
plt.plot(range(1, 4), train_rmse_errors[:3], label='TRAIN RMSE')
plt.plot(range(1, 4), test_rmse_errors[:3], label='TEST RMSE')
plt.xlabel("Polynomial Complexity")
plt.ylabel("RMSE")
plt.legend()

## Finalizing Model Choice

There are now 2 things we need to save, the Polynomial Feature creator AND the model itself. Let's explore how we would proceed from here:

1. Choose final parameters based on test metrics
2. Retrain on all data
3. Save Polynomial Converter object
4. Save model

In [ ]:
# Based on our chart, we select degree=2
final_poly_converter = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
final_model = LinearRegression()

In [ ]:
final_model.fit(final_poly_converter.fit_transform(X), y)

### Saving Model and Converter

In [ ]:
from joblib import dump, load

In [ ]:
dump(final_model, 'sales_poly_model.joblib') 

In [ ]:
dump(final_poly_converter,'poly_converter.joblib')

## Deployment and Predictions

### Prediction on New Data

Recall that we will need to **convert** any incoming data to polynomial data, since that is what our model is trained on. We simply load up our saved converter object and only call **.transform()** on the new data, since we're not refitting to a new data set.

**Our next ad campaign will have a total spend of 149 on TV, 22 on Radio, and 12 on Newspaper Ads, how many units could we expect to sell as a result of this?**

In [ ]:
loaded_poly = load('poly_converter.joblib')
loaded_model = load('sales_poly_model.joblib')

In [ ]:
ctv = np.array([149])
cradio = np.array([22])
cnewspaper = np.array([12])

campaign = pd.DataFrame({'tv': ctv, 'radio': cradio, 'newspaper': cnewspaper})

In [ ]:
campaign_poly = loaded_poly.fit_transform(campaign)

In [ ]:
campaign_poly

In [ ]:
final_model.predict(campaign_poly)